<a href='http://www.holoviews.org'><img src="assets/hv+bk.png" alt="HV+BK logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>11. Deploying Bokeh Apps</h2></div>

In the previous sections we discovered how to use a ``HoloMap`` to build a Jupyter notebook with interactive visualizations that can be exported to a standalone HTML file, as well as how to use ``DynamicMap`` and ``Streams`` to set up dynamic interactivity backed by the Jupyter Python kernel. However, frequently we want to package our visualization or dashboard for wider distribution, backed by Python but run outside of the notebook environment. Bokeh Server provides a flexible and scalable architecture to deploy complex interactive visualizations and dashboards, integrating seamlessly with Bokeh and with HoloViews.

For a detailed background on Bokeh Server see [the Bokeh user guide](http://bokeh.pydata.org/en/latest/docs/user_guide/server.html). In this tutorial we will discover how to deploy the visualizations we have created so far as a standalone Bokeh Server app, and how to flexibly combine HoloViews and ParamBokeh to build complex apps. We will also reuse a lot of what we have learned so far---loading large, tabular datasets, applying Datashader operations to them, and adding linked Streams to our app.

## A simple Bokeh app

The preceding sections of this tutorial focused solely on the Jupyter notebook, but now let's look at a bare Python script that can be deployed using Bokeh Server:

In [ ]:
with open('../apps/server_app.py', 'r') as f:
    print(f.read())

Step 1 of this app should be very familiar by now -- declare that we are using Bokeh to render plots, load some taxi dropoff locations, declare a Points object, Datashade them, and set some plot options.

At this point, if we were working with this code in a notebook, we would simply type ``shaded`` and let Jupyter's rich display support take over, rendering the object into a Bokeh plot and displaying it inline.  Here, step 2 adds the code necessary to do those steps explicitly:

- get a handle on the Bokeh renderer object using ``hv.renderer``
- create a Bokeh document from ``shaded`` by passing it to the renderer's ``server_doc`` method
- optionally, change some properties of the Bokeh document like the title.

This simple chunk of boilerplate code can be added to turn any HoloViews object into a fully functional, deployable Bokeh app!

## Deploying the app

Assuming that you have a terminal window open with the ``pyviz`` environment activated, in the ``../apps/`` directory, you can launch this app using Bokeh Server:

```
bokeh serve --show server_app.py
```

If you don't already have a favorite way to get a terminal, one way is to [open it from within Jupyter](../terminals/1), then make sure you are in the ``../apps`` directory, and activate the environment using ``source activate pyviz`` (or ``activate pyviz`` on Windows).

In [ ]:
# Exercise: Modify the app to display the pickup locations and add a tilesource, then run the app with bokeh serve
# Tip: Refer to the previous notebook


## Building an app with custom widgets

The above app script can be built entirely without using Jupyter, though we displayed it here using Jupyter for convenience in the tutorial.  Jupyter notebooks are also often helpful when initially developing such apps, allowing you to quickly iterate over visualizations in the notebook, deploying it as a standalone app only once we are happy with it. In this section we will combine everything we have learned so far including declaring of various parameters to control our visualization using a set of widgets.

We begin as usual with a set of imports:

In [ ]:
import holoviews as hv, geoviews as gv, param, parambokeh, dask.dataframe as dd

from colorcet import cm_n
from bokeh.document import Document
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY

hv.extension('bokeh', logo=False)

Next we once again load the Taxi dataset and define a tile source:

In [ ]:
usecols = ['dropoff_x', 'dropoff_y', 'pickup_x', 'pickup_y', 'dropoff_hour']
df = dd.read_parquet('../data/nyc_taxi_wide.parq/')
df = df[usecols].persist()

url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
tiles = gv.WMTS(url)
tile_options = dict(width=600,height=400,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)

Finally we will put together a complete dashboard with a number of parameters controlling our visualization, including controls over the alpha level of the tiles and the colormap as well as the hour of day and whether to plot dropoff or pickup location.

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha      = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap   = param.ObjectSelector(default=cm_n["fire"], objects=cm_n.values())
    hour       = param.Range(default=(0, 24), bounds=(0, 24))
    location   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])

    def make_view(self, x_range, y_range, **kwargs):
        map_tiles = tiles.opts(style=dict(alpha=self.alpha), plot=tile_options)
        points = hv.Points(df, kdims=[self.location+'_x', self.location+'_y'], vdims=['dropoff_hour'])
        if self.hour != (0, 24): points = points.select(dropoff_hour=self.hour)
        taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=self.colormap,
                               dynamic=False, x_range=x_range, y_range=y_range, width=600, height=400)
        return map_tiles * taxi_trips

explorer = NYCTaxiExplorer(name="NYC Taxi Trips")
dmap = hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])
plot = hv.renderer('bokeh').get_plot(dmap, doc=Document())
parambokeh.Widgets(explorer, view_position='right', callback=explorer.event, plots=[plot])

Now let's open the [text editor](../edit/apps/nyc_taxi/main.py) again and make this edit to a separate app, which we can then launch using Bokeh Server from the [terminal](../terminals/1).

In [ ]:
# Exercise: Note the differences between the server app and the app defined above
#           then add an additional parameter and plot

In [ ]:
# Exercise: Click the link below and edit the Jinja2 template to customize the app 

[Edit the template](../edit/apps/nyc_taxi/templates/index.html)

## Combining HoloViews with bokeh models

Now for a last hurrah let's put everything we have learned to good use and create a bokeh app with it. This time we will go straight to a [Python script containing the app](../edit/apps/player_app.py). If you run the app with ``bokeh serve --show ./apps/player_app.py`` from [your terminal](../terminals/1) you should see something like this:

<img src="./assets/tutorial_app.gif"></img>

This more complex app consists of several components:

1. A datashaded plot of points for the indicated hour of the daty (in the slider widget)
2. A linked ``PointerX`` stream,  to compute a cross-section
3. A set of custom Bokeh widgets linked to the hour-of-day stream

We have already covered 1. and 2. so we will focus on 3., which shows how easily we can combine a HoloViews plot with custom Bokeh models. We will not look at the precise widgets in too much detail, instead let's have a quick look at the callback defined for slider widget updates:

```python
def slider_update(attrname, old, new):
    stream.event(hour=new)
```

Whenever the slider value changes this will trigger a stream event updating our plots. The second part is how we combine HoloViews objects and Bokeh models into a single layout we can display. Once again we can use the renderer to convert the HoloViews object into something we can display with Bokeh:

```python
renderer = hv.renderer('bokeh')
plot = renderer.get_plot(hvobj, doc=curdoc())
```

The ``plot`` instance here has a ``state`` attribute that represents the actual Bokeh model, which means we can combine it into a Bokeh layout just like any other Bokeh model:

```python
layout = layout([[plot.state], [slider, button]], sizing_mode='fixed')
curdoc().add_root(layout)
```

In [ ]:
# Advanced Exercise: Add a histogram to the bokeh layout next to the datashaded plot
# Hint: Declare the histogram like this: hv.operation.histogram(aggregated, bin_range=(0, 20))
#       then use renderer.get_plot and hist_plot.state and add it to the layout


# Onwards

Although the code above is more complex than in previous sections, it's providing a huge range of custom types of interactivity, which if implemented in Bokeh alone would have required far more than a notebook cell of code.  Hopefully it is clear that arbitrarily complex collections of visualizations and interactive controls can be built from the components provided by HoloViews, allowing you to make simple analyses very easily and making it practical to make even quite complex apps when needed.  The [user guide](http://holoviews.org/user_guide), [gallery](http://holoviews.org/gallery/index.html), and [reference gallery](http://holoviews.org/reference) should have all the information you need to get started with all this power on your own datasets and tasks.  Good luck!